In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Directorio donde se encuentran los archivos CSV
csv_directory = './archive/'  # Asegúrate de especificar la ruta correcta

# Lista de nombres de archivos
file_names = [
    'dishwasher_53.csv',
    'boiler_226.csv',
    'air_purifier_293.csv',
]

# Hiperparámetros de la red LSTM
sequence_length = 10
num_features = 1
epochs = 10

for file_name in file_names:
    # Crear un DataFrame vacío para almacenar los datos
    df = pd.DataFrame()

    # Leer cada archivo CSV y agregarlo al DataFrame
    file_path = os.path.join(csv_directory, file_name)
    df = pd.read_csv(file_path)

    # Convertir la columna de timestamp en datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Establecer la columna de timestamp como el índice del dataframe
    df = df.set_index('timestamp')

    # Normalizar los datos de potencia
    scaler = StandardScaler()
    df['power'] = scaler.fit_transform(df['power'].values.reshape(-1, 1))

    # Crear secuencias de datos para entrenar la red LSTM
    data = []
    for i in range(len(df) - sequence_length):
        data.append(df.iloc[i:i+sequence_length])
    data = np.array(data)

    # Dividir los datos en conjuntos de entrenamiento y prueba
    train_size = int(0.8 * len(data))
    x_train = data[:train_size, :-1]
    y_train = data[:train_size, -1]
    x_test = data[train_size:, :-1]
    y_test = data[train_size:, -1]

    # Crear la red LSTM
    model = Sequential()
    model.add(LSTM(50, input_shape=(sequence_length-1, num_features)))
    model.add(Dense(num_features))
    model.compile(optimizer='adam', loss='mse')

    # Entrenar la red LSTM
    model.fit(x_train, y_train, epochs=epochs)

    # Realizar predicciones
    y_pred = model.predict(x_test)

    # Calcular el error cuadrático medio (MSE)
    mse = mean_squared_error(y_test, y_pred)

    print(f'Model for {file_name}:')
    print(f'Mean Squared Error: {mse}')

In [29]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Directorio donde se encuentran los archivos CSV
csv_directory = './archive/'  # Asegúrate de especificar la ruta correcta

# Lista de nombres de archivos
file_names = [
    'dishwasher_53.csv',
    'boiler_226.csv',
]

# Hiperparámetros de la red LSTM
sequence_length = 10
num_features = 1
epochs = 10

# Comprobar si el archivo del modelo existe
model_filename = 'modelo.h5'
if os.path.exists(model_filename):
    # Si el archivo del modelo existe, cargar el modelo
    model = load_model(model_filename)
else:
    # Si el archivo del modelo no existe, crear un nuevo modelo
    model = Sequential()
    model.add(LSTM(50, input_shape=(sequence_length-1, num_features)))
    model.add(Dense(num_features))
    model.compile(optimizer='adam', loss='mse')

for file_name in file_names:

    # Crear un DataFrame vacío para almacenar los datos
    df = pd.DataFrame()

    # Leer cada archivo CSV y agregarlo al DataFrame
    file_path = os.path.join(csv_directory, file_name)
    df = pd.read_csv(file_path)

    # Convertir la columna de timestamp en datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Establecer la columna de timestamp como el índice del dataframe
    df = df.set_index('timestamp')

    # Normalizar los datos de potencia
    scaler = StandardScaler()
    df['power'] = scaler.fit_transform(df['power'].values.reshape(-1, 1))

    # Crear secuencias de datos para entrenar la red LSTM
    data = []
    for i in range(len(df) - sequence_length):
        data.append(df.iloc[i:i+sequence_length])
    data = np.array(data)

    # Dividir los datos en conjuntos de entrenamiento y prueba
    train_size = int(0.8 * len(data))
    x_train = data[:train_size, :-1]
    y_train = data[:train_size, -1]
    x_test = data[train_size:, :-1]
    y_test = data[train_size:, -1]

    # Continuar entrenando el modelo sin reiniciar
    model.fit(x_train, y_train, epochs=epochs)

# Guardar el modelo entrenado
model.save('modelo.h5')

# Cargar el modelo en el futuro
modelo_cargado = load_model('modelo.h5')

Epoch 1/10
64800/64800 [==============================] - 440s 7ms/step - loss: 0.0016
Epoch 2/10
64800/64800 [==============================] - 434s 7ms/step - loss: 7.9517e-04
Epoch 3/10
16156/64800 [======>.......................] - ETA: 5:25 - loss: 8.6317e-04

In [ ]:
from sklearn.metrics import mean_squared_error

# Cargar el modelo en el futuro
modelo_cargado = load_model('model.h5')

# Realizar predicciones con el modelo cargado
# Asume que tienes un conjunto de datos de prueba x_test
y_pred = modelo_cargado.predict(x_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

# Medir la precisión del modelo
# Define un umbral (threshold) adecuado y compara las predicciones con los datos reales
threshold = 0.1  # Puedes ajustar este valor según tu problema
accuracy = np.mean(np.abs(y_pred - y_test) <= threshold * np.abs(y_test))
print(f'Accuracy: {accuracy}')

16200/16200 [==============================] - 121s 7ms/step
Mean Squared Error (MSE): 0.0009821110160464345
Accuracy: 0.9912210478801078
